In [ ]:
# Title: Void drift in a one-dimensional boiling canal #

In [ ]:
from trustutils import run 

# Purpose

The aim of this test case is to validate a series of drift-flux models against analytical solutions for a gas-liquid two-phase flow in a one-dimensional boiling canal. The relative velocity between gas and liquid phases is computed with different drift-flux models prior to being compared to their analytical solutions. 

Validation made by Corentin Reiss, report written by Claire Beauchesne. 


# Problem description
In this test case, a one-dimensional canal is filled with liquid sodium, which enters the canal at a temperature just below the boiling temperature, prior to being heated up by a volumetric heat source in the canal. As the liquid is heated up, the vapor void fraction increases from bottom to top. 

## Initial and boundary conditions
Liquid sodium enters the canal at a temperature of 879.5 °C and a velocity of 1 m/s. It is then heated all along the canal by a volumetric heat source of 2000000 W.m-3. The outlet pressure on the top side of the canal is set to 1 Pa. The mixture outlet temperature is set to the boiling temperature at 1 Pa, i.e. 879.9C°. 

The liquid initial temperature is set to 879.5 °C and its initial velocity to 1 m/s all along the canal. 

## Fluid properties
Both phases are assumed incompressible. Their properties are calculated from the analytical formulations reported in the technical note [1]. 
The sodium liquid phase properties are defined at a temperature of 800.0°C and a pressure of 1 bar. 
The sodium gas phase properties are defined at a temperature of 879.94°C and a pressure of 1 bar.


## Flow physics

The volumetric heat source increases the liquid temperature up to its vaporisation temperature and then turns into vapor. Both phases rise up to the outlet with a different velocity. 

Both phases are assumed strongly coupled but with a velocity desequilibrium so that the system of equations is made of three equations for mass, momentum and energy transport supplemented by an equation for the relative velocity between the two phases. The relative velocity between them may be modeled by different drift-flux velocity correlations. 

At the component scale, the gas velocity $v_g$ is given by a drift-flux correlation defined by two factors $C_0$ and $v_{g0}$ which depends on the chosen model (Ishii or constant drift-velocity):

$$v_g = C_0*(\alpha_g*v_g + \alpha_l*v_l) + v_{g0}$$

The relative velocity $u_r$ between the gas phase and the liquid phase is then obtained by the following expression: 

$$u_r = v_g - ( v_g - C_0*v_g*\alpha_g - v_{g0}  )/((1-\alpha_g)*C_0)$$

where the variables $\alpha_g$ and $v_g$ are the simulated void fraction and gas velocity. These formulations refers to the three test cases : "3eq\_vrnull", "3eq\_vrconstant", "3eq\_ishii".

In the last case, "3eq\_forces", the defined drift-velocity is valid at the local scale. In this case, the relative velocity is obtained after writting the force balance applied to the gas phase. Indeed, the dispersed gas phase is assumed in equilibrium in the liquid phase. Consequently, the force balance equates to zero and therefore allows a formulation of the relative velocity. The following algorithm gives the relative velocity:

    vr0 = 0
    vr1 = 5

    while (vr1-vr0>1.e-4):
        vrmed = (vr0+vr1)/2

        fi = 1. / 8 * C_d * 3 * alpha_g * (1-alpha_g) / Rb 
        fi *= (alpha_g*rhog+(1-alpha_g)*rhol) * vrmed**2
        if fi < 9.81*alpha_g*(1-alpha_g)*(rhol-rhog):
            vr0 = vrmed
        else : 
            vr1 = vrmed

    return vr0

where alpha\_g, rhog and rhol are the simulated void fraction, gas and liquid volumetric mass density. 


# Case setup 
## Grid

The geometry is represented by a 1 m-side square. Computations are performed using a 200 cells grid: 201 nodes in the z-direction, 2 in the x and y-directions. Finally, the mesh is polyedrized in order to use PolyMAC discretisation. 

## Model options

The multiphase equations solved are the equations of Pb_HEM with an imposed relative velocity between the two phases. The time scheme is the Euler implicite scheme.


In [ ]:
# Analytical solution for the relative velocity
# In the first two cases (constant drift velocity or Ishii and Hibiki models), the analytical calculation of the relative velocity is calculated as follows:
# alpha : gas void fraction
# vg : gas velocity
# C0 and vg0 are the drift-flux correlation parameters
def vitesse_relative_C0_dv0(alpha, vg, vg0, C0):
    return vg - ( vg - C0*vg*alpha - vg0  )/((1-alpha)*C0)

# For the last case (vitesse derive Forces), the determination of the relative velocity is based on a budget of forces along gravity:  
# alpha : gas void fraction
# rhol : liquid volumetric mass fraction
# Rb : bubble radius
# Cd : drag coefficient
def vr_FI_Bulles(alpha, rhol, rhog, Rb, C_d) :
    vr0 = 0
    vr1 = 5

    while (vr1-vr0>1.e-4):
        vrmed = (vr0+vr1)/2

        fi = 1. / 8 * C_d * 3 * alpha * (1-alpha) / Rb 
        fi *= (alpha*rhog+(1-alpha)*rhol) * vrmed**2
        if fi < 9.81*alpha*(1-alpha)*(rhol-rhog):
            vr0 = vrmed
        else : 
            vr1 = vrmed

    return vr0

In [ ]:
# definition of the calculation configurations

configs = [ "3eq_vrnull",
            "3eq_vrconstant",
            "3eq_ishii",
            "3eq_forces"
          ]
labels = [ "3eq_vr0",
           "3eq_vrconstant",
           "3eq_ishii",
           "3eq_forces"
          ]
correlations_deriv = ["Vitesse_relative derive_constante   { vg0_x 0 vg0_y 0 vg0_z 0 c0 1. }",
                      "Vitesse_relative derive_constante   { vg0_x 0 vg0_y 0 vg0_z 0.1 c0 1. }",
                      "vitesse_relative derive_ishii { subcooled_boiling  0 }",
                      "frottement_interfacial bulles   { coeff_derive .5 rayon_bulle 1.e-3 } vitesse_relative derive_forces { }"
                     ]

# Results
## Validation specific informations

* Version TRUST : 1.9.2
* Type of problem : 1D two-phase problem
* Discretization : PolyMAC
* Time Scheme: Euler implicite
* Momentum convection scheme: amont
* Solving of equations: Masse_Multiphase, QDM_Multiphase, Energie_Multiphase, a formulation of the relative velocity
* Location: /share/Validation/Rapports_automatiques/Multiphase/CMFD/Verification1D_drift-flux_models
* Generated test cases: 
	* 3eq_vrnull.data
    * 3eq_vrconstant.data
    * 3eq_ishii.data
    * 3eq_forces.data

In [ ]:
# Generation of the four calculation cases
import os, sys
from string import Template

force_recalculation = True 

n_raf = 1
title_raf = [1]

def GenerateInputFile(name,substitutions_dict):
    with open(f"{run.BUILD_DIRECTORY}/jdd.data", "r") as file: 
        filedata = Template(file.read())
    result = filedata.substitute(substitutions_dict)
    with open(f"{run.BUILD_DIRECTORY}/{name}.data", "w") as file:
        file.write(result)

if force_recalculation or not os.path.exists(f'{run.BUILD_DIRECTORY}'):
    run.reset()
    run.defaultSuite_ = run.TRUSTSuite(runPrepare=False)

    for c in range(len(configs)) :
        if True :
            if True :
                folder_run = f"{configs[c]}"
                os.system(f'mkdir {run.BUILD_DIRECTORY}/{folder_run}')
            
                if True :
                    name_run = f"{folder_run}"

                    substitutions_dict = {"correlations_deriv" : correlations_deriv[c] # ,                                          "sondes" : str_sonde ,
                                          # "schema" : schema[c] ,
                                          # "polyedriser" : polyedriser[c]
                                         }
                    GenerateInputFile(name_run,substitutions_dict)

                    os.system(f'mv {run.BUILD_DIRECTORY}/{name_run}.data {run.BUILD_DIRECTORY}/{folder_run}')
                    run.addCase(f"./{folder_run}", f"{name_run}.data")
    
    run.runCases()

In [ ]:
display(run.tablePerf())

## Plot Data

Simulation results and analytical solutions are plotted on the figure below. For all four cases, the void fraction, the relative velocity and the pressure profiles along the canal are plotted. As we can see, there is an excellent agreement between simulated and analytical relative velocities for all models. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib
import copy
import time
from IPython.display import display
import math
from trustutils import visit
from trustutils.jupyter import plot

def loadText(data, index_column=0, nb_column=-1, transpose=True, dtype="float", skiprows=0):   
    if nb_column == -1:
        nb = None
    else:
        nb = index_column + nb_column
    try:
        if transpose:
            matrix = np.loadtxt(data, dtype=dtype, skiprows=skiprows).T[index_column:nb]
        else:
            matrix = np.loadtxt(data, dtype=dtype, skiprows=skiprows)[index_column:nb]
    except:
        matrix = np.loadtxt(data, dtype=dtype, skiprows=skiprows)                
    return matrix

In [ ]:
t_mesure = -1

fig = plt.figure(figsize = (15, 15))
fig.suptitle("Comparison of the numerical results against the analytical solutions\n", fontsize = 18)
axs = fig.subplots(3, 4)

for c in range(len(configs)) :
    folder_run = f"{configs[c]}"
    absc = np.linspace(5e-4, 1-5e-4, 1000)
    alphaSim  = np.array(loadText(f"{run.BUILD_DIRECTORY}/{folder_run}/{folder_run}_ALPHA_GAZ_SODIUM.son"))[1::,t_mesure]
    vgSim     = np.array(loadText(f"{run.BUILD_DIRECTORY}/{folder_run}/{folder_run}_VITESSE_GAZ_SODIUM.son"))[3::3,t_mesure]
    vlSim     = np.array(loadText(f"{run.BUILD_DIRECTORY}/{folder_run}/{folder_run}_VITESSE_LIQUIDE_SODIUM.son"))[3::3,t_mesure]
    TgSim     = np.array(loadText(f"{run.BUILD_DIRECTORY}/{folder_run}/{folder_run}_TEMPERATURE.son"))[2::2,t_mesure]
    TlSim     = np.array(loadText(f"{run.BUILD_DIRECTORY}/{folder_run}/{folder_run}_TEMPERATURE.son"))[1::2,t_mesure]
    rhogSim   = np.array(loadText(f"{run.BUILD_DIRECTORY}/{folder_run}/{folder_run}_MASSE_VOLUMIQUE.son"))[2::2,t_mesure]
    rholSim   = np.array(loadText(f"{run.BUILD_DIRECTORY}/{folder_run}/{folder_run}_MASSE_VOLUMIQUE.son"))[1::2,t_mesure]
    PSim      = np.array(loadText(f"{run.BUILD_DIRECTORY}/{folder_run}/{folder_run}_PRESSION.son"))[1::,t_mesure]
    
    axs[0+(c//4)*3,  c%4].plot(absc, alphaSim, label = "Simulation results")
    axs[1+(c//4)*3,  c%4].plot(absc, vgSim-vlSim, label = "Simulation results")
    axs[2+(c//4)*3,  c%4].plot(absc, PSim - PSim[-1], label = "Simulation results")
    
    if (c%4==0) : 
        ur_th = np.linspace(0, 0, 1000) #relative velocity equates to 0 since 0c=1 and vg0=0
    elif (c%4==1) : 
        ur_th = vitesse_relative_C0_dv0(alphaSim, vgSim, .1, 1.) 
    elif (c%4==2) : 
        C0_loc = 1.2 + (1.0 - 1.2) * (rhogSim / rholSim)**.5 
        sigma_sodium = 0.2405 * pow(1 - (TgSim+273.15) / 2503.7, 1.126)
        sigma_eau = 5.e-3
        dv0_loc = 2.**.5 * ( (rholSim- rhogSim) * 9.81 * sigma_sodium / rholSim / rholSim)**0.25 * (1.0 - alphaSim)**1.75
        ur_th = vitesse_relative_C0_dv0(alphaSim, vgSim, dv0_loc, C0_loc )
        
    elif (c%4==3) :
        ur_th = [  vr_FI_Bulles(alphaSim[i], rholSim[i], rhogSim[i], 1.e-3, 0.5) for i in range(1000)]


    axs[1+(c//4)*3,  c%4].plot(absc, ur_th, 'k--', label = "Analytical solution")
    
    P_calc = np.linspace(PSim[-1], PSim[-1], 1000)
    for i in range(998, -1, -1) :
        P_calc[i] = P_calc[i+1] + 9.81*1.e-3*0.5*(alphaSim[i]*rhogSim[i]+alphaSim[i+1]*rhogSim[i+1]+(1-alphaSim[i])*rholSim[i]+(1-alphaSim[i+1])*rholSim[i+1])
    axs[2+(c//4)*3,  c%4].plot(absc, P_calc- P_calc[-1], 'k--', label = "Hydrostatic pressure")
    if c==3 :
        PSim4 = PSim
    
    axs[0+(c//4)*3,  c%4].legend(fontsize = 10)
    axs[1+(c//4)*3,  c%4].legend(fontsize = 10)
    axs[2+(c//4)*3,  c%4].legend(fontsize = 10)

    axs[0+(c//4)*3,  c%4].set_ylabel(r"Void fraction $\alpha$", fontsize = 14)
    axs[1+(c//4)*3,  c%4].set_ylabel(r"Relative velocity $u_g-u_l$ (m/s)", fontsize = 14)
    axs[2+(c//4)*3,  c%4].set_ylabel(r"Pressure $P-P_{out}$ (Pa)", fontsize = 14)
    for i in range(3):
        axs[i+(c//4)*3,  c%4].set_xlabel(r"Height $z$ (m)", fontsize = 14)
        axs[i+(c//4)*3,  c%4].set_title(labels[c], fontsize = 14)

plt.tight_layout()
plt.show()

## Conclusion
All four problems reach a steady state. 
The results show an excellent agreement between simulated and analytical relative velocities for all models. 

## References
[1] DEN/DANS/DM2S/STMF/LMES/RT/12-018/A

## Data files

### Keyword
In the following generic data file, the keyword correlations_deriv is overwritten by the proper correlation as detailed below in order to generate the four data file: 

	* "Vitesse_relative derive_constante { vg0_x 0 vg0_y 0 vg0_z 0 c0 1.0 }" 
       in 3eq_vrnull.data
    * "Vitesse_relative derive_constante { vg0_x 0 vg0_y 0 vg0_z 0 c0 1.0 }" 
       in 3eq_vrconstant.data
    * "vitesse_relative derive_ishii { subcooled_boiling  0 }" 
       in 3eq_ishii.data
    * "frottement_interfacial bulles { coeff_derive .5 rayon_bulle 1.e-3 } ,  
       vitesse_relative derive_forces { }"
       in 3eq_forces.data

### Generic data file

```
# PARALLEL OK #
dimension 3
Domaine dom

# BEGIN MESH #
mailler  dom
{
 pave carre
 {
     origine  0.0  0.0 0.0
     nombre_de_noeuds  2  2 201
     longueurs 1 1 1
 }
 {
     bord  bas       Z = 0.0    0.0 <= X <= 1.0 0.0 <= Y <= 1.
     bord  haut      Z = 1.0    0.0 <= X <= 1.0 0.0 <= Y <= 1.
     bord  boundary  X = 0.0    0.0 <= Y <= 1.0 0.0 <= Z <= 1.
     bord  boundary  X = 1.0    0.0 <= Y <= 1.0 0.0 <= Z <= 1.
     bord  boundary  Y = 0.0    0.0 <= X <= 1.0 0.0 <= Z <= 1. 
     bord  boundary  Y = 1.0    0.0 <= X <= 1.0 0.0 <= Z <= 1. 
 }
}
polyedriser dom
# END MESH #

polymac_p0  dis option_covimac {  }


Schema_euler_implicite sch
Read sch
{
 tinit 0
 tmax 3
 seuil_statio 1e-4
 solveur ice
 {
     criteres_convergence { alpha 1e-5  }
     solveur petsc cli { }
     seuil_convergence_implicite 1e30
 }
}

Pb_HEM pb
Associate  pb dom
Associate  pb sch
Discretize pb dis

Read pb
{
 Milieu_composite
 {
     liquide_sodium Fluide_sodium_liquide {  P_ref 100000.0 T_ref 800 }
     gaz_sodium Fluide_sodium_gaz { P_ref 100000.0 T_ref 879.9430097752451 }
     saturation_sodium saturation_sodium { P_ref 100000.0 T_ref 879.9430097752451 }
 }
 correlations
 {
 Gravite Multiphase champ_uniforme 3 0 0 -9.81
 $correlations_deriv
 }
 QDM_Multiphase
 {
     solveur_pression petsc cli_quiet { -pc_type hypre -pc_hypre_type boomeramg }
     convection { amont }
     diffusion  { negligeable }
     initial_conditions
     {
         vitesse  Champ_Fonc_xyz dom 6 0 0 0 0 1 1
         pression Champ_Fonc_xyz dom 1 100000
     }
     boundary_conditions
     {
         boundary symetrie
         bas  frontiere_ouverte_vitesse_imposee_sortie champ_front_fonc_t 6 0 0 0 0 1 1
         haut frontiere_ouverte_pression_imposee champ_front_uniforme 1 100000
     }
 sources
 {  }
 }
 Masse_Multiphase
 {
     initial_conditions { alpha Champ_Fonc_xyz dom 2 1 0 }
     convection { amont }
     boundary_conditions
     {
         boundary paroi
         bas  frontiere_ouverte a_ext Champ_Front_Uniforme 2 1 0
         haut frontiere_ouverte a_ext Champ_Front_Uniforme 2 1 0
     }
     sources {  }
 }
 Energie_Multiphase
 {
     diffusion { negligeable }
     convection { amont }
     initial_conditions { temperature Champ_Uniforme 2 879.5 879.9430097752451 }
     boundary_conditions
     {
         boundary paroi_adiabatique
         bas    frontiere_ouverte T_ext Champ_Front_Uniforme 2 879.5 879.9430097752451
         haut   frontiere_ouverte T_ext Champ_Front_Uniforme 2 879.5 879.9430097752451
     }
     sources
     {
         travail_pression ,
         puissance_thermique Champ_composite 2
         {
             champ_uniforme 1 2000000.0
             champ_uniforme 1 0
         } 
     }
 }
 Post_processing
 {
 sondes
 {
     alpha_gaz_sodium alpha_gaz_sodium periode 1.e-2 segment 1000 
                                                         0.5 0.5 0.0005 0.5 0.5 0.9995 
     vitesse_liquide_sodium vitesse_liquide_sodium periode 1.e-2 segment 1000  
                                                         0.5 0.5 0.0005 0.5 0.5 0.9995 
     vitesse_gaz_sodium vitesse_gaz_sodium periode 1.e-2 segment 1000  
                                                         0.5 0.5 0.0005 0.5 0.5 0.9995 
     temperature temperature periode 1.e-2 segment 1000 0.5 0.5 0.0005 0.5 0.5 0.9995 
     masse_volumique masse_volumique periode 1.e-2 segment 1000  
                                                         0.5 0.5 0.0005 0.5 0.5 0.9995 
     pression pression periode 1.e-2 segment 1000 0.5 0.5 0.0005 0.5 0.5 0.9995 
 }

     Format lml 
     fields dt_post 1e-2
     {
         vitesse_liquide_sodium elem
         vitesse_gaz_sodium elem
         alpha_gaz_sodium elem
         pression elem
         temperature elem
         energie_interne elem
         enthalpie elem
     }
 }
}
Solve pb
End
 ```

In [ ]:
#from trustutils import run 
#run.dumpDataset(f"{run.BUILD_DIRECTORY}/jdd.data", list_Trust_user_words="")